In [24]:
import pandas as pd
import string
import re
import nltk
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import words
from nltk import pos_tag
from nltk.tokenize import word_tokenize

In [25]:
# data crawl từ tiki
crawl_df = pd.read_csv('output.csv', encoding='utf-8')

In [26]:
# data cleaned từ file txt
clean_df = pd.read_csv('cleaned_data.csv', encoding='utf-8')

In [27]:
crawl_df.shape

(96132, 2)

In [28]:
clean_df.shape

(14976, 2)

In [29]:
# Concatenate các DataFrame
df = pd.concat([crawl_df, clean_df])
df

,Label,Name
0,mobile,điện thoại ai samsung galaxy s24 ultra 12gb/25...
1,mobile,điện thoại samsung galaxy a15 lte (8gb/128gb)-...
2,mobile,điện thoại samsung galaxy a05 (4gb/128gb) - đã...
3,mobile,điện thoại samsung galaxy a05s (4gb/128gb) - đ...
4,mobile,apple iphone 13
...,...,...
14971,máy làm lạnh nước,máy làm lạnh nước orion rks1500f
14972,bàn ghế trang điểm,bàn phấn mdf 1m2 7706
14973,bàn phím,bàn phím rapoo v31
14974,sàn gỗ,sàn gỗ morser 12 ly 501


In [30]:
df = df.dropna()

In [31]:
import unicodedata
# Dạng dựng sẵn (precomposed form, NFC - Normalization Form C): Các ký tự có dấu được biểu diễn bằng một ký tự duy nhất. Ví dụ: "ế" được biểu diễn bằng một ký tự duy nhất.
def normalize_unicode(df, columns, form='NFC'):
    for column in columns:
        df[column] = df[column].apply(lambda x: unicodedata.normalize(form, x) if isinstance(x, str) else x)
    return df

# Chuẩn hóa các cột 'Name' và 'Label' trong cả hai DataFrame
columns_to_normalize = ['Name', 'Label']
df = normalize_unicode(df, columns_to_normalize)

In [32]:
# Tính số lượng dòng cho mỗi nhóm Label
label_counts = df['Label'].value_counts()
label_counts

bao đựng ốp lưng điện thoại               342
bàn ghế phòng khách                       187
giày da nam                               185
ghế văn phòng                             179
bộ bàn ghế cafe                           175
                                         ... 
gps hệ thống định vị vệ tinh trên ô tô      1
đồng phục gia đình                          1
tấm lót bồn cầu tấm lót bệ xí               1
vách tắm cabin tắm                          1
thép cây thép thanh                         1
Name: Label, Length: 2047, dtype: int64

In [33]:
# Lấy nhưng quan sát có label count >= 30 còn những label > 50 sẽ down sample xuống 50
filtered_dfs = []

for label, count in label_counts.items():
    if count >= 30:
        if count > 50:
            # Lấy ngẫu nhiên 50 mẫu nếu count > 50
            sampled_df = df[df['Label'] == label].sample(n=50, random_state=1)
        else:
            # Lấy tất cả các mẫu nếu count <= 50 nhưng >= 30
            sampled_df = df[df['Label'] == label]
        filtered_dfs.append(sampled_df)

# Kết hợp các DataFrame lại
result_df = pd.concat(filtered_dfs)
df = result_df

In [34]:
df['Label'].value_counts()

bao đựng ốp lưng điện thoại    50
máy dán miệng cốc              50
thuốc tiểu đường               50
thời trang cho vật nuôi        50
vải lọc chống tĩnh điện        50
                               ..
hộp tỳ tay trên ô tô           33
lắc tay nam                    31
phục trang nam khác            31
giường trẻ em                  31
bàn cafe café                  30
Name: Label, Length: 1930, dtype: int64

In [35]:
# Text norm udts
from underthesea import text_normalize
# Áp dụng hàm text_normalize lên cột 'Name'
df['Name'] = df['Name'].apply(text_normalize)

In [36]:
# Hàm làm sạch văn bản
def name_clean_text(text):
    # Chuyển đổi thành chữ thường
    text = text.lower()

    # Loại bỏ ký tự khoảng trống
    text = re.sub(r'\u200b', '', text)

    # Loại bỏ chuỗi chứa số
    text = re.sub(r'\S*\d+\S*', '', text)

    # Loại bỏ dấu câu và ký tự đặc biệt
    text = re.sub(f"[{string.punctuation}]", "", text)

    # Loại bỏ từ dừng
    stop_words = set(['và', 'của', 'là', 'có', 'trong', 'cho', 'bằng', 'làm', 'bởi', 'với'])
    words = [word for word in text.split() if word not in stop_words]

    return ' '.join(words)

# Áp dụng hàm làm sạch cho các cột 'Name' và 'Label'
df['Name'] = df['Name'].apply(name_clean_text)

In [37]:
df = df.reset_index(drop=True)

In [38]:
df['Label'].value_counts()

bao đựng ốp lưng điện thoại    50
máy dán miệng cốc              50
thuốc tiểu đường               50
thời trang cho vật nuôi        50
vải lọc chống tĩnh điện        50
                               ..
hộp tỳ tay trên ô tô           33
lắc tay nam                    31
phục trang nam khác            31
giường trẻ em                  31
bàn cafe café                  30
Name: Label, Length: 1930, dtype: int64

In [39]:
org_df = df

In [47]:
org_df

,Label,Name
0,bao đựng ốp lưng điện thoại,ốp lưng xiaomi redmi a j case tản nhiệt đen
1,bao đựng ốp lưng điện thoại,ốp vân da cao cấp dành samsung galaxy
2,bao đựng ốp lưng điện thoại,ốp lưng uniq outfitter vintage iphone màu kem msp
3,bao đựng ốp lưng điện thoại,ốp lưng tráng gương sony xperia
4,bao đựng ốp lưng điện thoại,ốp dẻo vu da iphone plus north
...,...,...
96137,bàn cafe café,bàn cafe việt nhất vndt
96138,bàn cafe café,bàn cafe mặt vuông gỗ bạch đàn
96139,bàn cafe café,bộ bàn ghế
96140,bàn cafe café,bàn cà phê


In [40]:
from underthesea import chunk
# danh từ (N), đại từ (P), trạng từ (R), tính từ (A), động từ (V), và ký tự đặc biệt (CH)
def filter_nouns(text):
    chunks = chunk(text)
    nouns = [word for word, pos, chunk_tag in chunks if len(chunk_tag) >=2 and chunk_tag[2] in ('N','V','A')]
    return ' '.join(nouns)

# Áp dụng hàm filter_nouns lên cột 'Name'
df['Name'] = df['Name'].apply(filter_nouns)

In [55]:
filter_nouns('ốp dẻo vu da iphone plus north')

'ốp dẻo vu da iphone plus north'

### Liệt kê các từ hiếm mà không có trong từ điển tiếng việt và tiếng anh

In [56]:
import json
#source từ điển: https://github.com/undertheseanlp/dictionary/tree/master
#Đường dẫn đến tệp words.txt
path = 'words.txt'

#Tạo một tập hợp để lưu trữ các giá trị unique của 'text'
vietnamese_unique_texts = set()

#Đọc từng dòng từ tệp và xử lý JSON
with open(path, 'r', encoding='utf-8') as file:
   for line in file:
       # Chuyển đổi dòng văn bản JSON thành dictionary
       data = json.loads(line.strip())
       # Tách giá trị 'text' thành các từ riêng lẻ và thêm vào tập hợp
       words = data['text'].split()
       for word in words:
           vietnamese_unique_texts.add(word.lower())

In [57]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import words
from nltk import pos_tag
from nltk.tokenize import word_tokenize

# Tải các tài nguyên cần thiết (chạy lệnh này một lần nếu chưa có)
#nltk.download('wordnet')
#nltk.download('averaged_perceptron_tagger')
#nltk.download('words')

In [58]:
# Load English words
english_words = set(words.words())

def is_english_noun(word):
    # Check if the word is in the set of English words
    if word not in english_words:
        return False

    # Get part-of-speech tag for the word
    pos = pos_tag([word])[0][1]

    # Check if the word is a noun
    return pos in ['NN', 'NNS', 'NNP', 'NNPS']

In [59]:
def is_vietnamese_word(word):
    return True if word in vietnamese_unique_texts else False

In [60]:
from collections import Counter
# Tạo từ điển tần suất từ
word_counts = Counter(" ".join(df['Name']).split())

# Đặt ngưỡng tần suất từ để xác định từ hiếm gặp
threshold = 5 # Từ xuất hiện ít hơn hoặc bằng ngưỡng này sẽ được coi là hiếm gặp

# Lọc các từ hiếm gặp dựa trên ngưỡng tần suất và yêu cầu

rare_words = [word for word, count in word_counts.items() if count <= threshold and not is_english_noun(word) and not is_vietnamese_word(word)]
rare_words = set(rare_words)
print("Các từ hiếm gặp:", rare_words)
print(len(rare_words))

Các từ hiếm gặp: {'luxuna', 'vacuumejector', 'numbers', 'yg', 'eziplug', 'rizin', 'jlab', 'ztab', 'gbm', 'dots', 'rac', 'kippzonen', 'wood’s', 'fhumic', 'doremidi', 'gastro', 'ciscorn', 'sáchxa', 'buccotherm', 'mizuno', 'bookends', 'infusing', 'gluter', 'miguel', 'dreamlife', 'chicago', 'sukia', 'aquafluide', 'piega', 'vhp', 'knts', 'livik', 'dymax', 'bspt', 'psitẩu', 'sganoal', 'fins', 'ecosophy', 'hkd', 'scarves', 'codie', 'pinpop', 'crimping', 'jeju', 'dau', 'jinluda', 'ussa', 'camshield', 'pz', 'luseta', 'acroots', 'neru', 'xenyx', 'leeasy', 'arirang', 'pít', 'resorts', 'aromatherapy', 'vitolize', 'ăng', 'jumu', 'binzel', 'convertible', 'koleos', 'minee', 'erhytry', 'alixx', 'toneup', 'sloggi', 'uxg', 'quetrầm', 'têt', 'mahalo', 'goii', 'cemboard', 'roel', 'chie', 'gosutráng', 'icc', 'bluetoothopticalusbchỉnh', 'ulnatech', 'mramerican', 'maccoffee', 'ovany', 'comotomo', 'trắngrăng', 'arianna', 'emsrf', 'iot', 'olizen', 'bejo', 'dexe', 'writing', 'đỏsize', 'classeq', 'butiq', 'caful

### Loại bỏ các từ hiếm trên

In [61]:
# Hàm loại bỏ các từ hiếm gặp từ một chuỗi văn bản
def remove_single_chars_and_rare_words(text, rare_words):
    # Kết hợp loại bỏ các từ đơn ký tự và các từ hiếm gặp
    return " ".join([word for word in text.split() if len(word) > 1 and word not in rare_words])

# Áp dụng hàm loại bỏ từ hiếm gặp cho cột 'Name'
df.loc[:, 'Name'] = df['Name'].apply(lambda x: remove_single_chars_and_rare_words(x, rare_words))

In [62]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96142 entries, 0 to 96141
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   96142 non-null  object
 1   Name    96142 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [63]:
df = df.reset_index(drop=True)

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96142 entries, 0 to 96141
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   96142 non-null  object
 1   Name    96142 non-null  object
dtypes: object(2)
memory usage: 1.5+ MB


In [65]:
df['Label'].value_counts()

bao đựng ốp lưng điện thoại    50
máy dán miệng cốc              50
thuốc tiểu đường               50
thời trang cho vật nuôi        50
vải lọc chống tĩnh điện        50
                               ..
hộp tỳ tay trên ô tô           33
lắc tay nam                    31
phục trang nam khác            31
giường trẻ em                  31
bàn cafe café                  30
Name: Label, Length: 1930, dtype: int64

In [68]:
df

,Label,Name
0,bao đựng ốp lưng điện thoại,ốp lưng xiaomi redmi case tản nhiệt đen
1,bao đựng ốp lưng điện thoại,ốp vân da cao cấp dành samsung galaxy
2,bao đựng ốp lưng điện thoại,ốp lưng outfitter vintage iphone màu kem msp
3,bao đựng ốp lưng điện thoại,ốp lưng tráng gương sony
4,bao đựng ốp lưng điện thoại,ốp dẻo vu da iphone plus north
...,...,...
96137,bàn cafe café,bàn cafe việt nhất vndt
96138,bàn cafe café,bàn cafe mặt vuông gỗ bạch đàn
96139,bàn cafe café,bộ bàn ghế
96140,bàn cafe café,bàn cà phê


In [82]:
# Tạo một DataFrame mới chỉ chứa các giá trị NaN hoặc rỗng trong cột 'Name'
nan_rows = df[df['Name'].isna() | (df['Name'] == '')]

# Nhóm theo cột 'Label' và đếm số lượng NaN hoặc rỗng trong cột 'Name'
nan_count_per_label = nan_rows.groupby('Label').size()

print("Số lượng giá trị NaN hoặc rỗng trong cột 'Name' cho từng 'Label':")
print(nan_count_per_label)

Số lượng giá trị NaN hoặc rỗng trong cột 'Name' cho từng 'Label':
Label
bộ liên hoàn trong nhà       1
công tắc cảm ứng             1
hóa chất công nghiệp         3
máy ghi âm                   1
mực in                       1
receiver                     3
son                          1
sữa tắm                      1
thiết bị kiểm soát ra vào    1
vật liệu chống thấm          3
đồ dùng sinh hoạt khác       1
dtype: int64


In [84]:
# Xóa các quan sát có giá trị NaN hoặc rỗng trong cột 'Name'
df_cleaned = df.dropna(subset=['Name'])
df_cleaned = df_cleaned[df_cleaned['Name'] != '']

print("DataFrame sau khi xóa các quan sát có giá trị NaN hoặc rỗng trong cột 'Name':")
df_cleaned

DataFrame sau khi xóa các quan sát có giá trị NaN hoặc rỗng trong cột 'Name':


,Label,Name
0,bao đựng ốp lưng điện thoại,ốp lưng xiaomi redmi case tản nhiệt đen
1,bao đựng ốp lưng điện thoại,ốp vân da cao cấp dành samsung galaxy
2,bao đựng ốp lưng điện thoại,ốp lưng outfitter vintage iphone màu kem msp
3,bao đựng ốp lưng điện thoại,ốp lưng tráng gương sony
4,bao đựng ốp lưng điện thoại,ốp dẻo vu da iphone plus north
...,...,...
96137,bàn cafe café,bàn cafe việt nhất vndt
96138,bàn cafe café,bàn cafe mặt vuông gỗ bạch đàn
96139,bàn cafe café,bộ bàn ghế
96140,bàn cafe café,bàn cà phê


In [85]:
df_cleaned.to_csv('tiki_data.csv', encoding='utf-8', index=False)